In [1]:
import pandas as pd
import numpy as np
import preprocessor as pre
from datetime import datetime

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
ds=pd.read_json('../dataset/Tweet-Fact-Not-Fact.jsonl')
ds.head()

,id,text,labels
0,591974898139987968,RT @Economynext: #SriLanka to fly emergency me...,1
1,591979323164233729,India committed in aid to Nepal: President Pra...,-1
10,591977175768698881,RT @iamamkay: need all out support @nytimes: N...,-1
100,591996410792321024,RT @MERCYMalaysia: MERCY Malaysia will be resp...,-1
1000,592204836038311937,RT @AmeriCares: #Nepal is a very poor country ...,-1


In [4]:
id2tweet={}
for ix in range(len(ds)):
    pre.set_options(pre.OPT.URL, pre.OPT.EMOJI,pre.OPT.SMILEY,pre.OPT.MENTION,pre.OPT.HASHTAG)
    tweet=pre.clean(ds.text[ix].replace("\n"," "))
    id2tweet[ds.id[ix]]=tweet

In [5]:
all_ids=list(id2tweet.keys())

In [6]:
def check(ix):
    doc=nlp(id2tweet[ix])
    print (id2tweet[ix])
    pnoun=0
    num=0
    for token in doc:
            if (token.pos_ == "PROPN") :
                print ('Pnoun--> ',token)
                pnoun+=1
            if (token.pos_ == "NUM"):
                print ('num--> ',token)
                num+=1
    print (doc.ents)
            

In [7]:
id2features={}
i=0
for ix in list(id2tweet.keys()):
    i=i+1
    if(i%2000==0):
        print(i)
    doc=nlp(id2tweet[ix])
    pnoun=0
    num=0
    for token in doc:
        if (token.pos_ == "PROPN") :
            pnoun+=1
        if (token.pos_ == "NUM"):
            num+=1
    id2features[ix]=[pnoun,len(doc.ents),num]

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000


In [8]:
l=[]
for ix in all_ids:
    l.append([ix]+id2features[ix])
df=pd.DataFrame(l,columns=['id','pnouns','ent','num'])

In [9]:
id2tweet[df.id[4]]
check(df.id[4])

India sends 39 team, 2 dogs and 3 tonnes equipment to Nepal Army for rescue operations: Indian Embassy in
Pnoun-->  India
num-->  39
num-->  2
num-->  3
Pnoun-->  Nepal
Pnoun-->  Army
Pnoun-->  Indian
Pnoun-->  Embassy
(India, 39, 2, 3 tonnes, Nepal Army, Indian, Embassy)


In [11]:
df.head()

,id,pnouns,ent,num
0,591974898139987968,1,1,0
1,591979323164233729,9,5,0
2,591984387526234112,2,0,0
3,591984010714157056,0,0,0
4,591974595525128195,5,7,3


In [18]:
sorted(list(df.pnouns))[-1]

18

In [12]:
df_ent=df[['id','ent']]
df_ent=df_ent.sort_values('ent',ascending=False).reset_index(drop=True)
df_ent.head()

,id,ent
0,593343223852113920,12
1,592902646555631616,11
2,594505741337112576,11
3,592701043030495232,11
4,592679376761835520,10


In [13]:
file=open('ent_100.txt','w')
txt=''
for ix in range(100):
    temp=df_ent.id[ix]
    txt+=f'{temp} {id2tweet[temp]}\n'
file.write(txt)
file.close()

In [28]:
df.head()

,id,pnouns,ent,num,score
0,591974898139987968,1,1,0,0.1
1,591979323164233729,9,5,0,0.9
2,591984387526234112,2,0,0,0.2
3,591984010714157056,0,0,0,0.0
4,591974595525128195,5,7,3,0.8


In [29]:
check(591984010714157056)

Quake wake-up call for govt, need better building tech: Eert
()


In [14]:
for ix in range(10):
    check(df_ent.id[ix])
    print ('-'*100)

Currently working rescue Army : CHINA, INDIA,FRANCE,ISREAL,TURKEY,GERMANY,USA,UK,UAE,SRILANKA,PAKISTAN,CANADA
Pnoun-->  Army
Pnoun-->  CHINA
Pnoun-->  INDIA
Pnoun-->  FRANCE
Pnoun-->  ISREAL
Pnoun-->  TURKEY
Pnoun-->  GERMANY
Pnoun-->  USA
Pnoun-->  UK
Pnoun-->  UAE
Pnoun-->  SRILANKA
Pnoun-->  PAKISTAN
Pnoun-->  CANADA
(Army, CHINA, INDIA, FRANCE, ISREAL, TURKEY, GERMANY, USA, UK, UAE, PAKISTAN, CANADA)
----------------------------------------------------------------------------------------------------
Earthquake Helpline: West Bengal: 1070, MHA. 23093564/66, NDMA. 26701728/29, 9868881801 Indian Embassy in Kathmandu: +977 9851107021/35141
Pnoun-->  Earthquake
Pnoun-->  Helpline
Pnoun-->  West
Pnoun-->  Bengal
num-->  1070
Pnoun-->  MHA
num-->  23093564/66
Pnoun-->  NDMA
num-->  26701728/29
num-->  9868881801
Pnoun-->  Indian
Pnoun-->  Embassy
Pnoun-->  Kathmandu
num-->  977
num-->  9851107021/35141
(West Bengal, 1070, MHA, 23093564/66, NDMA, 26701728/29, 9868881801, Indian, Embassy, K

In [15]:
df.pnouns=df.pnouns.apply(lambda x: min(x,5))
df.ent=df.ent.apply(lambda x: min(x,5))
df.num=df.num.apply(lambda x: min(x,5))

In [17]:
df.head()

,id,pnouns,ent,num
0,591974898139987968,1,1,0
1,591979323164233729,5,5,0
2,591984387526234112,2,0,0
3,591984010714157056,0,0,0
4,591974595525128195,5,5,3


In [22]:
comb_score=[]
for ix in range(len(df)):
    try:
        comb_score.append(0.1*df.pnouns[ix]+0.1*df.num[ix])
    except:
        print (df)

In [23]:
df['score']=comb_score

In [27]:
df.head()

,id,pnouns,ent,num,score
0,591974898139987968,1,1,0,0.1
1,591979323164233729,9,5,0,0.9
2,591984387526234112,2,0,0,0.2
3,591984010714157056,0,0,0,0.0
4,591974595525128195,5,7,3,0.8


In [25]:
df_sorted=df.sort_values('score',ascending=False).reset_index(drop=True)

In [26]:
df_sorted.head()

,id,pnouns,ent,num,score
0,596387549796442112,18,4,3,2.1
1,592679376761835520,10,10,10,2.0
2,594501061924167680,10,10,10,2.0
3,594081370835562496,12,9,7,1.9
4,592523896357695488,6,6,13,1.9


In [34]:
txt=''
for ix in range(100):
    txt+=id2tweet[df_sorted.id[ix]]+'\n'
print (txt)
file=open('best_fire.txt','w')
file.write(txt)
file.close()

3Novices:Nepal Earthquake: Man Rescued After 80 Hours in Collapsed Building With 3 Bodies A 28-year-old man spent over 80 hours with three …
RT : MEA opens 24 hour Control Room for queries regarding the Nepal Earthquake. Numbers: +91 11 2301 2113 +91 11 2301 4104 +91 11…
Green earthquake alert (Magnitude 4.6M, Depth:10km) in Nepal 26/04/2015 22:32 UTC, 13197114 people within 100km. (vía GDACS)
RT : BJPDelhiState: RT MEAIndia: In action again. 24 hour control room 4 Nepal Earthquake. Numbers +91 11 2301 2113 +91 11 …
RT : 4.6 earthquake, 57km NNE of Tambo, Peru. Apr 26 16:10 at epicenter (14m ago, depth 70km).
26 Dec 1861 Earthquake in Egion, Yunani 26 Mar 1872 Earthquake in Owens Valley, USA 26 Aug 1896 Earthquake in...
MEA opens 24 hour Control Room for queries regarding tragic Nepal Earthquake. Numbers: +91 11 2301 2113 +91 11...
RT : 4.6 earthquake, 76km SSE of Bitung, Indonesia. May 2 13:23 at epicenter (19m ago, depth 85km).
4.9 earthquake! Fri May 08 06:17:33 GMT+05:45 2015 near 